In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd 
import numpy as np 

In [10]:
## load all the traind model,scaler and encoders
model = load_model('churn_model.h5')
# load the encoder and scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('ohe_geo.pkl', 'rb') as f:
    encoder = pickle.load(f)
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

In [3]:
# take an example input
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [ ]:
# onehot encode the Geography feature (robust to sparse/dense outputs and sklearn API)
geo_val = input_data['Geography']
try:
    # transform expects 2D input: provide a single-row 2D list
    geo_transformed = encoder.transform([[geo_val]])
    # If transform returns a sparse matrix, use .toarray(); otherwise fall back to np.asarray
    try:
        geo_array = geo_transformed.toarray()
    except Exception:
        geo_array = np.asarray(geo_transformed)
    # Get column names (sklearn version compatibility)
    if hasattr(encoder, 'get_feature_names_out'):
        cols = encoder.get_feature_names_out(['Geography'])
    elif hasattr(encoder, 'get_feature_names'):
        cols = encoder.get_feature_names(['Geography'])
    else:
        cols = [f'Geography_{i}' for i in range(geo_array.shape[1])]
    geo_df = pd.DataFrame(geo_array, columns=cols)
except ValueError as e:
    # Unknown category or other transform error: fallback to pandas.get_dummies
    print('Warning during encoder.transform:', e)
    geo_df = pd.get_dummies(pd.Series([geo_val]), prefix='Geography')
# display the resulting one-hot dataframe
geo_df

c:\udemy_DL_churn_modeling\churn\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [9]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
#encode gender
input_df['Gender'] = label_encoder.transform(input_df['Gender'])

In [13]:
input_df = pd.concat([input_df.drop('Geography', axis= 1), geo_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
#scale the numerical features
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
# predict churn
prediction = model.predict(input_scaled)
prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step


array([[0.06539926]], dtype=float32)

In [ ]:
churn_probability = prediction[0][0]
print(f"Churn Probability: {churn_probability:.4f}")


Churn Probability: 0.0654
